# Mine Reddit Data with PRAW and save in a long format excel dataframe
Hunter Priniski (priniski@ucla.edu)

__Note.__ Before running this script, I advise you glance at the PRAW documentation: https://praw.readthedocs.io/en/latest/

In [ ]:
import praw
import json
import os
import re

In [ ]:
#Follow Oauth set up here https://redditclient.readthedocs.io/en/latest/oauth/
reddit = praw.Reddit(client_id='',
                     client_secret='',
                     password='',
                     user_agent=''
                     username='')

In [ ]:
#Replies to a given comment will be represented as a list of reply IDs. Useful for constructing discussion trees
#You can remove this function if you perfer the _replies data structure returned by the Reddit API 
def get_replies(comment):
    replies = comment['_replies']._comments

    replies_list = []

    if len(replies) > 0:

        for reply in replies:
            replies_list.append(reply.id)

        return replies_list

    else:
        return []

In [ ]:
#Extract the complete set of comments from a discussion (Note. Here, replies are stored as a list of IDs)
def get_comments(submission):
    
    comments = []
    com = submission.comments.replace_more(limit=submission.num_comments)
    com_tree = submission.comments[:]

    while com_tree:
        comment = com_tree.pop(0)                    
        com_tree.extend(comment.replies)
        comments.append(comment)

    data_dicts = []
    
    for comment in comments:
        comment_dict = vars(comment)
        comment_body = comment_dict['body']
        data_dict = {field:comment_dict[field] for field in comment_dict.keys()}
        data_dict['_replies'] = get_replies(comment_dict)
        
        try: 
            data_dict['author'] = vars(data_dict['author'])['name']
        
        except TypeError: 
            continue
            
        data_dicts.append(data_dict)

    return data_dicts

Here, we store the discussion data as a JSON object in a directory labeled `reddit_data`. I stringify some objects to allow for this. All JSON objects will be saved in memory.

In [ ]:
#specify the name of the subreddit as a string
subreddit_name = ''
subreddit = reddit.subreddit(subreddit_name)

# you can change which type of posts you would like to collect here
for submission in subreddit.top(limit = 1000):
    
    submission_dict = vars(submission)
    author = submission_dict['author']
    post = {field:submission_dict[field] for field in submission_dict.keys()}
    post['_comments'] = get_comments(submission)
    if post['author']: 
        post['author'] = vars(post['author'])['name']
    else: 
        post['author'] = 'NA'
    post['_reddit'] = '_reddit'
    post['subreddit'] = subreddit_name
    
    for comment in post['_comments']:
        comment['_submission'] = comment['_submission'].id
        comment['_reddit'] = '_reddit'
        comment['subreddit'] = subreddit_name
        
    with open('reddit_data/'+ post['name'] +'.json', 'w') as outfile:
        print("Wrote %s as JSON File" % post['name'])
        json.dump(post, outfile)

In [ ]:
#Load all JSON objects saved in memory into a single list
posts = [] 
directory = os.fsencode('reddit_data/')     

for file in os.listdir(directory): 
    filename = os.fsdecode(file)    
    if filename.endswith(".json"):  
        
        with open('reddit_data/' + filename, 'r') as json_file:
            post = json.load(json_file)
            posts.append(post)  

We will create a list of lists using the JSON objects saved in our `reddit_data` folder. Each sublist will be a row in the final dataframe, and the metalist will be the complete dataframe. In the final dataframe, each row represents a single comment in the discussion. I like to create trim dataframes without all of the metadata. However, if there is data that you would like to include, you can index the discussion like a Python dictionary. Hence, `posts[0].keys()` will return the dicsusison level metadata and `posts[0]['_comment'].keys()` will return the comment level metadata.

In [ ]:
df = []

#create a trim dataframe of reddit posts
for post in posts:
    subreddit = post['subreddit']
    title = post['title']
    selftext = post['selftext']
    selftext_html = post['selftext_html']
    name = post['name']
    author = post['author']
    score = post['score']
    
    
    for comment in post['_comments']:
        
        comment_replies = comment['_replies']
        comment_body = comment['body']
        comment_html = comment['body_html']    
        comment_name = comment['name']
        comment_author = comment['author']
        comment_score = comment['ups']
    
        replies = comment['_replies']
        
        row = [subreddit, title, selftext, selftext_html, name, author, score,
              comment_replies, comment_body, comment_html, comment_name, comment_author, comment_score]
        
        df.append(row)

In [ ]:
#label the objects in the row list. This will become our dataframe header, so use clear labels
cols = ['subreddit', 'title', 'selftext', 'selftext_html', 'name', 'author', 'score',
              'comment_replies', 'comment_body', 'comment_html', 'comment_name', 'comment_author', 'comment_score']

In [ ]:
#Turn the list of lists to a pandas dataframe
DF = pd.DataFrame(df, columns = cols)

In [ ]:
#Save the dataframe to an excel file
DF.to_excel('reddit_data.xlsx', index = False)